# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 14

### Elementos do Grupo

Número: 55853    Nome: Madalena Rodrigues    
Número: 56897    Nome: Pedro Almeida    
Número: 56935    Nome: Rómulo Nogueira     

## Representação dos estados

Definimos um classe **PacmanEstado**, para representar o estado, que contém dentro dela a informação que não é estática, ou seja os cinco atributos dinâmicos:
* posicao: (x,y) correspondem às coordenadas do pacman em que, x é a linha e y a coluna;
* pontuacao: n é a pontuação atual;
* patilhas: (((x1,y1), T1), ((x2,y2), T2)...) - em que T é o tipo da pastilha
* custos: ((x1,y1), (x2,y2), ... ,(xn,yn)) em que os tuplos são as coordenadas das casas já visitadas pelo pacman
* t: tempo decorrido 

Estado inicial:
* PacmanEstado.posicao -> (x,y) - coordenadas definidas pelo utilizador
* PacmanEstado.pontuacao -> 0
* PacmanEstado.pastilhas -> (((x1,y1), T1), ((x2,y2), T2)...) - em que T é o tipo da pastilha
* PacmanEstado.custos -> ()
* PacmanEstado.t -> 0

Ações: [N,S,E,O] conforme a existência de obstáculos, em que:
* N corresponde ao deslocamento (-1,0)
* S corresponde ao deslocamento (1,0)
* E corresponde ao deslocamento (0,1)
* O corresponde ao deslocamento (0,-1)

Sucessor: Nova localização do pacman. Caso coincida com uma pastilha atualiza a pontuação bem como o conjunto de casas visitadas e o instante t

Teste de objetivo: PacmanEstado.pontuacao igual ao valor 'goal' definido pelo utilizador
* goal_test((*, goal)) -> * = estado qualquer possível

Foi também necessário redefinir o método **__equals__** bem como o **__hash__** para que fosse possível comparar os estados, por se tratarem de objetos.


### *Exemplo de representação de um estado:*
```
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  C  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  D  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  N  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
```

Estado: s1
* s1.posicao = (1,1)
* s1.pontuacao = 0
* s1.pastilhas = (((1,2),N), ((7,3),N), ((5,4),D), ((3,7),C))
* s1.custo = ((2,1),(3,1),(2,1),(1,1))
* s1.t = 4


Ações: [S,E]

Possíveis sucessores: s2 e s2'
- s2:
    - s2.posicao = (2,1);
    - s2.pontuacao = 0;
    - s2.pastilhas = (((1,2),N), ((7,3),N), ((5,4),D), ((3,7),C));
    - s2.custos = ((2,1),(3,1),(2,1),(1,1),(2,1));
    - s2.t = 5.

- s2':
    - s2'.posicao = (1,2);
    - s2'.pontuacao = 1;
    - s2'.pastilhas = (((7,3),N), ((5,4),D), ((3,7),C));
    - s2'.custos = ((2,1),(3,1),(2,1),(1,1),(1,2));
    - s2'.t = 5.

Considerámos ainda que a casa inicial não representa qualquer custo, e por isso, ao revisitá-la pela primeira vez tem custo de 1.


## Formulação do problema

Formulámos um problema de navegação, numa grelha 2D em que algumas células são obstáculos impassíveis e outras contém pastilhas que o Pacman deve "comer" de forma a ganhar pontos.

No caso do nosso problema o objetivo é obter uma pontuação x, por isso na função **goal_test()** verificamos se  o objetivo foi atingido; as **actions()** serão as direções para onde o pacman se poderá mover, tendo em conta os obstáculos existentes; o **result()** devolve o estado atualizado, com a nova posição, pontuação atual, a lista de pastilhas que ainda existem no tabuleiro disponíveis, os custos e o tempo decorrido; por fim, o **path_cost()**, considera os custos associados à deslocação do pacman no tabuleiro, sendo que o custo de passar numa casa aumenta 1 a cada vez que esta é visitada.

##### *Algumas funções auxiliares utilizadas*:

In [1]:
def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

def exec(p,estado,accoes):
    """Executa uma sequência de acções numa lista, devolvendo o estado e custo resultantes da operação."""
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
   
    p.display(estado)
    print('Custo:',custo)
    print('Goal?',p.goal_test(estado))
    print('Pontuação: ', estado.pontuacao)
    return (estado,custo)

In [2]:
from searchPlus import *
import timeit

##### *Definição da classe estado*:

In [3]:
class PacmanEstado:
    def __init__(self, posicao, pontuacao, pastilhas, custos, t):
        """O construtor especifica o estado inicial, constituído pela posição, 
        pontuação, pastinhas disponíveis, custo e tempo gasto"""
        self.posicao = posicao
        self.pontuacao = pontuacao
        self.pastilhas = pastilhas
        self.custos = custos
        self.t = t

    def __eq__(self, other):
        """Define em que circunstâncias os dois estados são considerados iguais.
        Necessário para os algoritmos de procura em grafo.
        """
        return self.posicao == other.posicao and \
            self.pastilhas == other.pastilhas and \
            self.pontuacao == other.pontuacao and \
            self.custos == other.custos and \
            self.t == other.t

    def __hash__(self):
        """Retorna valores inteiros a objetos"""
        return hash(self.posicao) ^ hash(self.pontuacao) ^ hash(self.pastilhas) ^ hash(self.custos) ^ hash(self.t)
            
    def __str__(self): 
        """Retorna a representação dos objetos em forma de string"""
        return f"estado: posição: {self.posicao}, pontuacao: {self.pontuacao}"

    def __repr__(self):
        """Neste caso não existe interesse em retornar diferntes outputs
        que na função __str__, por isso __repr__ chama a função __str__"""
        return str(self)

    def __lt__(self, other):
        """Faz a comparação de estados para o algoritmo do custo uniforme"""
        return True

In [5]:
class PacmanPastilhas(Problem):

    def __init__(self, pacman=(1,1), goal=1, pastilhas=(), obstacles=quadro(0,0,10) | line(2,2,1,0,5) | line(2,3,0,1,5), dim=10):
        """Construtor da classe, que acede aos dados da classe Problem e 
        adiciona os parâmetros correspondentes aos obstáculos e à dimensão 
        do tabuleiro, que podem ou não ser fornecidos pelo utilizadr."""
        super().__init__(PacmanEstado(pacman, 0, pastilhas, (), 0), goal)
        self.obstacles = obstacles
        self.dim = dim

    def actions(self, state):
        """As acções executáveis neste estado."""
        res = []
        moves = [('N',(-1,0)),('S',(1,0)),('O',(0,-1)),('E',(0,1))]
        for m in moves:
            novaPos = (state.posicao[0] + m[1][0], state.posicao[1] + m[1][1])
            if novaPos not in self.obstacles:
                res.append(m[0])
        return res

    def result(self, state, action):
        """Aplica a acção ao estado"""
        # calcular nova posicao
        anterior_posicao = state.posicao
        nova_posicao = ()
        
        if action == 'N':
            nova_posicao = (anterior_posicao[0] - 1, anterior_posicao[1])
        elif action == 'S':
            nova_posicao = (anterior_posicao[0] + 1, anterior_posicao[1])
        elif action == 'O':
            nova_posicao = (anterior_posicao[0], anterior_posicao[1] - 1)
        elif action == 'E':
            nova_posicao = (anterior_posicao[0], anterior_posicao[1]  + 1)

        # calculo do novo instante t
        novo_t = state.t + 1

        # calcular pontucao / remover pastilhas
        nova_pontuacao = state.pontuacao
        novas_pastilhas = list(state.pastilhas)
        posicoes_pastilhas = list(map(lambda l : l[0] , state.pastilhas))
        
        if nova_posicao in posicoes_pastilhas:
            i = posicoes_pastilhas.index(nova_posicao)
            pastilha = novas_pastilhas.pop(i)[1]

            if pastilha == "N":
                nova_pontuacao += 1
            elif pastilha == "D":
                nova_pontuacao += max(0, 5-novo_t)
            elif pastilha == "C":
                nova_pontuacao += novo_t

        # atualizar o custo
        novos_custos = list(state.custos)
        novos_custos.append(nova_posicao)
        
        return PacmanEstado(nova_posicao, nova_pontuacao, tuple(novas_pastilhas), tuple(novos_custos), novo_t)

    def goal_test(self, state):
        """Verifica se o objetivo (goal) foi atingido e retorna o resultado
        dessa verificação"""
        return state.pontuacao >= self.goal
    

    def path_cost(self, c, state1, action, state2):
        """Retorna o custo de um caminho de solução que chega ao state2 vindo de
        state1 depois da ação, assumindo custo c para chegar até state1. Se para o 
        problema o caminho não importar, esta função irá apenas olhar para state2. 
        Se o caminho importar, irá considerar c e talvez state1 e ação."""
        novo_custo = state2.custos.count(state2.posicao)
        return c + novo_custo

    def display(self, state):
        """Faz print do estado atual"""
        sort_obs = sorted(self.obstacles)
        posicoes_pastilhas = list(map(lambda l : l[0] , state.pastilhas))
        
        for x in range(self.dim):
            for y in range(self.dim):
                if (x,y) in sort_obs:
                    print(" = ", end='')
                elif (x,y) in posicoes_pastilhas:
                    i = posicoes_pastilhas.index((x,y))
                    print(" " + state.pastilhas[i][1] + " ", end='')
                elif (x,y) == state.posicao:
                    print(" @ ", end='')
                else:
                    print(' . ', end='')
            print('')

    def display_trace(self,d,plan):
        """Faz print do caminho que o Pacman percorreu até atingir o seu estado final"""
        path = []
        st = self.initial
        pastilhas = ()

        for a in plan:
            st = self.result(st,a)
            path.append(st.posicao)
            pastilhas = st.pastilhas
        
        sort_obs = sorted(self.obstacles)
        posicoes_pastilhas = list(map(lambda l : l[0] , pastilhas))
        path = list(dict.fromkeys(path))
       
        for i in range(d):
            for j in range(d):
                if self.initial.posicao ==(i,j):
                    print(" @ ", end='')

                elif (i,j) == path[-1]:
                    print(" * ", end='')

                elif (i,j) in sort_obs:
                    print(" = ", end='')

                elif (i,j) in posicoes_pastilhas:
                    g = posicoes_pastilhas.index((i,j))
                    print(" " + pastilhas[g][1] + " ", end ='')

                elif (i,j) in path:
                    print(" + ", end='')

                else:
                    print(" . ", end='')
            print('')

## Criação de estados e do problema

Criamos um mundo cercado 10 x 10, em que o Pacman começa na posição (5,1) e temos pastilhas nas posições: 
* Normais: (3,7), (7,1)
* Crescimento: (1,6), (7,3)
* Desgaste: (8,2)

E o objetivo de atingir uma pontuação de 7 pontos

In [6]:
l = line(2,2,1,0,5) 
c = line(2,3,0,1,5)
fronteira = quadro(0,0,10)
pastilhas = (((1,6), "C"), 
             ((3,7), "N"),
             ((8,2), "D"),
             ((7,3), "C"),
             ((7,1), "N"))


p1 = PacmanPastilhas(pacman= (5,1) ,goal= 7, pastilhas=pastilhas)
custo = 0
print("O estado inicialmente encontra-se da seguinte forma: ", p1.initial)
p1.display(p1.initial)
print("O objetivo é a atingir a pontuação de : " + str(p1.goal) + " pontos")
print("Começa com o custo de : ", custo)
print("Corresponde ao estado final? ", p1.goal_test(p1.initial))
print("As ações possíveis são: ", p1.actions(p1.initial))

O estado inicialmente encontra-se da seguinte forma:  estado: posição: (5, 1), pontuacao: 0
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  C  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  N  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  @  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  N  .  C  .  .  .  .  .  = 
 =  .  D  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
O objetivo é a atingir a pontuação de : 7 pontos
Começa com o custo de :  0
Corresponde ao estado final?  False
As ações possíveis são:  ['N', 'S']


Em seguida vemos então uma primeira iteração, em que o Pacman se desloca para baixo (S)

In [7]:
# Iter1
s1 = p1.result(p1.initial, "S")
print("Aplicando a ação 'S' ficamos com o", s1)
p1.display(s1)
custo = p1.path_cost(custo, p1.initial, "S", s1)
print("Com o custo de : ", custo)
print("Corresponde ao estado final? ", p1.goal_test(s1))
print("As ações possíveis são: ", p1.actions(s1))

Aplicando a ação 'S' ficamos com o estado: posição: (6, 1), pontuacao: 0
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  C  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  N  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  @  =  .  .  .  .  .  .  = 
 =  N  .  C  .  .  .  .  .  = 
 =  .  D  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
Com o custo de :  1
Corresponde ao estado final?  False
As ações possíveis são:  ['N', 'S']


Na segunda iteração, o Pacman desloca-se novamente para baixo (S) apanhando a pastilha que se encontra em (7,1)

In [8]:
# Iter2
print()

s2 = p1.result(s1, "S")
print("Aplicando a ação 'S' ficamos com o", s2)
p1.display(s2)
custo = p1.path_cost(custo, s1, "S", s2)
print("Com o custo de : ", custo)
print("Corresponde ao estado final? ", p1.goal_test(s2))
print("As ações possíveis são: ", p1.actions(s2))


Aplicando a ação 'S' ficamos com o estado: posição: (7, 1), pontuacao: 1
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  C  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  N  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  @  .  C  .  .  .  .  .  = 
 =  .  D  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
Com o custo de :  2
Corresponde ao estado final?  False
As ações possíveis são:  ['N', 'S', 'E']


Na terceira iteração, o Pacman desloca-se para a direita (E)

In [9]:
# Iter3
print()

s3 = p1.result(s2, "E")
print("Aplicando a ação 'E' ficamos com o :", s3)
p1.display(s3)
custo = p1.path_cost(custo, s2, "E", s3)
print("Com o custo de : ", custo)
print("Corresponde ao estado final? ", p1.goal_test(s3))
print("As ações possíveis são: ", p1.actions(s3))


Aplicando a ação 'E' ficamos com o : estado: posição: (7, 2), pontuacao: 1
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  C  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  N  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  @  C  .  .  .  .  .  = 
 =  .  D  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
Com o custo de :  3
Corresponde ao estado final?  False
As ações possíveis são:  ['S', 'O', 'E']


Na quarta iteração, o Pacman desloca-se para a baixo (S) apanhando a pastilha que se encontra em (8,2)

In [10]:
# Iter4
print()

s4 = p1.result(s3, "S")
print("Aplicando a ação 'S' ficamos com o:", s4)
p1.display(s4)
custo = p1.path_cost(custo, s3, "S", s4)
print("Com o custo de : ", custo)
print("Corresponde ao estado final? ", p1.goal_test(s4))
print("As ações possíveis são: ", p1.actions(s4))


Aplicando a ação 'S' ficamos com o: estado: posição: (8, 2), pontuacao: 2
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  C  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  N  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  C  .  .  .  .  .  = 
 =  .  @  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
Com o custo de :  4
Corresponde ao estado final?  False
As ações possíveis são:  ['N', 'O', 'E']


Na quinta iteração, o Pacman desloca-se para a cima (N) 

In [11]:
# Iter5
print()

s5 = p1.result(s4, "N")
print("Aplicando a ação 'N' ficamos com o:", s5)
p1.display(s5)
custo = p1.path_cost(custo, s4, "S", s5)
print("Com o custo de : ", custo)
print("Corresponde ao estado final? ", p1.goal_test(s5))
print("As ações possíveis são: ", p1.actions(s5))


Aplicando a ação 'N' ficamos com o: estado: posição: (7, 2), pontuacao: 2
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  C  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  N  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  @  C  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
Com o custo de :  6
Corresponde ao estado final?  False
As ações possíveis são:  ['S', 'O', 'E']


Na sexta e última iteração, o Pacman desloca-se para a direita (E), apanhando a última pastilha que precisa para atingir o objetivo em (7,4)

In [12]:
# Iter6
print()

s6 = p1.result(s5, "E")
print("Aplicando a ação 'E' ficamos com o:", s6)
p1.display(s6)
custo = p1.path_cost(custo, s5, "E", s6)
print("Com o custo de : ", custo)
print("Corresponde ao estado final? ", p1.goal_test(s6))


Aplicando a ação 'E' ficamos com o: estado: posição: (7, 3), pontuacao: 8
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  C  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  N  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  @  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  .  = 
 =  =  =  =  =  =  =  =  =  = 
Com o custo de :  7
Corresponde ao estado final?  True


#### Exemplo de execução com o a função exec()

In [13]:
l = line(2,2,1,0,5) 
c = line(2,3,0,1,5)
fronteira = quadro(0,0,10)
pastilhas = (((1,2),"N"), 
             ((3,7),"C"),
             ((5,4),"D"),
             ((7,3),"N"),
             ((8,8),"D"))

p2 = PacmanPastilhas(pacman=(1,1),goal=2,pastilhas=pastilhas,obstacles=fronteira | l | c ,dim=10)
print("Estado inicial:")
p2.display(p2.initial)
print("Estado final:")
exec(p2, p2.initial, ['S', 'N', 'E', 'O', 'S', 'S', 'S', 'S', 'S', 'S' ,'E', 'E'])

Estado inicial:
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  C  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  D  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  N  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 
Estado final:
 =  =  =  =  =  =  =  =  =  = 
 =  .  .  .  .  .  .  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  C  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  D  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  @  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 
Custo: 14
Goal? True
Pontuação:  2


(estado: posição: (7, 3), pontuacao: 2, 14)

## Teste de procura de solução
Vamos testar a procura de uma solução no seguinte problema:

In [14]:
l = line(2,2,1,0,5) 
c = line(2,3,0,1,5)
fronteira = quadro(0,0,10)

pastilhas = (((1,2),"N"), 
             ((3,7),"C"),
             ((5,4),"C"),
             ((7,3),"N"),
             ((8,8),"D"))

p = PacmanPastilhas(pacman=(1,1),goal=12,pastilhas=pastilhas,obstacles=fronteira | l | c ,dim=10)
p.display(p.initial)

 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  C  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  C  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  N  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


### Profundidade Primeiro

#### Em ávore:

In [15]:
def depth_first_tree_search(problem):
    """Search the deepest nodes in the search tree first."""
    return tree_search(problem, Stack())

In [16]:
start = timeit.default_timer()
resultado  = depth_first_tree_search(p)
print("Solução Profundidade-primeiro (árvore) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim, resultado.solution()) # Porque a diimensão não é sempre 10 e varia

KeyboardInterrupt: 

Reparamos que o algoritmo de **profundidade primeiro** na versão árvore, não encontrou uma solução para este problema.

Isto acontece porque este não é completo, ou seja, pode ficar preso num loop infinito. É o que acontece neste caso.

#### Em grafo:

In [17]:
def depth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, Stack())

In [18]:
start = timeit.default_timer()
resultado = depth_first_graph_search(p)
print("Solução Profundidade-primeiro (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

KeyboardInterrupt: 

Reparamos que, também neste algoritmo, não é possível encontrar uma solução para o problema.

Neste caso, não é possível encontrar a solução porque ao comparar dois estados nunca irão existir dois exatamente iguais, uma vez que o custo irá ser sempre diferente em ambos.

### Largura-primeiro

#### Em árvore

In [19]:
def breadth_first_tree_search(problem):
    """Search the shallowest nodes in the search tree first."""
    return tree_search(problem, FIFOQueue())

In [20]:
start = timeit.default_timer()
resultado = breadth_first_tree_search(p)
print("Solução Largura-primeiro (árvore) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

Solução Largura-primeiro (árvore) com custo 11:
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'N', 'N', 'E']
Time:  0.5311269000000038
Podemos ver o rasto do plano
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  +  =  =  =  =  =  =  .  = 
 =  +  =  .  .  .  .  C  .  = 
 =  +  =  .  .  .  .  .  .  = 
 =  +  =  +  *  .  .  .  .  = 
 =  +  =  +  .  .  .  .  .  = 
 =  +  +  +  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


#### Em grafo:

In [21]:
def breadth_first_graph_search(problem):
    """Search the deepest nodes in the search tree first."""
    return graph_search(problem, FIFOQueue())

In [22]:
start = timeit.default_timer()
resultado = breadth_first_graph_search(p)
print("Solução Largura-primeiro (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

Solução Largura-primeiro (grafo) com custo 11:
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'N', 'N', 'E']
Time:  4.419901399999993
Podemos ver o rasto do plano
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  +  =  =  =  =  =  =  .  = 
 =  +  =  .  .  .  .  C  .  = 
 =  +  =  .  .  .  .  .  .  = 
 =  +  =  +  *  .  .  .  .  = 
 =  +  =  +  .  .  .  .  .  = 
 =  +  +  +  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


No algoritmo de **Largura Primeiro**, é possivel observar que a procura em árvore é mais rápida que a procura em grafo. Isto acontece porque, o algoritmo da largura em grafo é mais complexo, porém se se tratassem de tabuleiros mais complicados a procura em grafo teria um melhor resultado, pois esta descarta as possibilidades que já foram expandidas.

Ainda assim, ao contrário dos algoritmos de profundidade primeiro, o algoritmo de largura primeiro consegue encontrar uma solução por ser um algortimo completo.

### Aprofundamento progressivo

#### Em árvore:

In [23]:
def iterative_deepening_search(problem):
    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, depth)
        if result != 'cutoff':
            return result

In [24]:
start = timeit.default_timer()
resultado = iterative_deepening_search(p)
print("Solução Aprofundamento-progressivo (árvore) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

Solução Aprofundamento-progressivo (árvore) com custo 11:
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'N', 'N', 'E']
Time:  0.03249859999999671
Podemos ver o rasto do plano
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  +  =  =  =  =  =  =  .  = 
 =  +  =  .  .  .  .  C  .  = 
 =  +  =  .  .  .  .  .  .  = 
 =  +  =  +  *  .  .  .  .  = 
 =  +  =  +  .  .  .  .  .  = 
 =  +  +  +  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


#### Em grafo:

In [25]:
def graph_limited_search(problem, frontier,lim):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    """
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        if node.depth < lim:
            frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return 'cutoff'

def depth_limited_graph_search(problem, depth):
    return graph_limited_search(problem,Stack(),depth)

def iterative_deepening_plus_graph_search(problem):
    for depth in range(sys.maxsize):
        result = depth_limited_graph_search(problem, depth)
        if result != 'cutoff':
            return result

In [26]:
start = timeit.default_timer()
resultado = iterative_deepening_plus_graph_search(p)
print("Solução aprofundamento progressivo (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

Solução aprofundamento progressivo (grafo) com custo 11:
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'N', 'N']
Time:  0.0613101999999941
Podemos ver o rasto do plano
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  +  =  =  =  =  =  =  .  = 
 =  +  =  .  .  .  .  C  .  = 
 =  +  =  .  .  .  .  .  .  = 
 =  +  =  .  *  .  .  .  .  = 
 =  +  =  .  +  .  .  .  .  = 
 =  +  +  +  +  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


Este algoritmo apresenta caminhos diferentes em árvore e em grafo. Isso deve-se ao facto dos nós não serem revisitados.

### Custo uniforme

O algoritmo do custo uniforme é aquele que independentemente da sua execução ser em árvore ou em grafo, encontra sempre a **solução ótima**.

#### Em árvore:
Tentámos fazer uma implementação da procura em árvore de modo a conseguir comparar com o grafo, porém não sabemos se o raciocínio foi bem feito.
Uma vez que a grande diferença entre os algoritmos em grafo e em árvore é o uso dos nós expandidos e visitados, retirámos o **expanded** de modo a poder ver revisitas e re-expansões dos nós.

In [27]:
def best_first_tree_search(problem, f):
    f = memoize(f, 'f')
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = PriorityQueue(min, f)
    frontier.append(node)
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        for child in node.expand(problem):
            if child not in frontier:
                frontier.append(child)
            elif child in frontier:
                incumbent = frontier[child]
                if f(child) < f(incumbent):
                    del frontier[incumbent]
                    frontier.append(child)
    return None

def uniform_cost_tree_search(problem):
    return best_first_tree_search(problem, lambda node: node.path_cost)


In [28]:
start = timeit.default_timer()
resultado = uniform_cost_tree_search(p)
print("Solução Custo-uniforme (árvore) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

Solução Custo-uniforme (árvore) com custo 11:
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'N', 'N']
Time:  0.06658400000000597
Podemos ver o rasto do plano
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  +  =  =  =  =  =  =  .  = 
 =  +  =  .  .  .  .  C  .  = 
 =  +  =  .  .  .  .  .  .  = 
 =  +  =  .  *  .  .  .  .  = 
 =  +  =  .  +  .  .  .  .  = 
 =  +  +  +  +  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


#### Em grafo: 

In [29]:
def uniform_cost_search(problem):
    return best_first_graph_search(problem, lambda node: node.path_cost)

In [30]:
start = timeit.default_timer()
resultado = uniform_cost_search(p)
print("Solução Custo-uniforme (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

Solução Custo-uniforme (grafo) com custo 11:
['S', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'N', 'N']
Time:  0.06810519999999087
Podemos ver o rasto do plano
 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  +  =  =  =  =  =  =  .  = 
 =  +  =  .  .  .  .  C  .  = 
 =  +  =  .  .  .  .  .  .  = 
 =  +  =  .  *  .  .  .  .  = 
 =  +  =  .  +  .  .  .  .  = 
 =  +  +  +  +  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


Podemos reparar que o algoritmo ignora por completo a pastilha que se encontra a **E** (à direita) quando está na posição inicial pois isso implicaria aumentar em 3 o custo e de forma desnecessária visto que existe um caminho com menor custo que atinge também a pontuação final.
É de notar que se alterármos o goal para 13 este deixaria de ser o caminho ótimo visto que não chegaria à pontuação desejada.
Portanto ele optaria, como podemos ver a seguir por outro caminho

In [32]:
p = PacmanPastilhas(pacman=(1,1),goal=13,pastilhas=pastilhas,obstacles=fronteira | l | c ,dim=10)
p.display(p.initial)

start = timeit.default_timer()
resultado = uniform_cost_search(p)
print("Solução Custo-uniforme (grafo) com custo", str(resultado.path_cost)+":")
print(resultado.solution())
stop = timeit.default_timer()
print('Time: ', stop - start)
print('Podemos ver o rasto do plano')
p.display_trace(p.dim,resultado.solution())

 =  =  =  =  =  =  =  =  =  = 
 =  @  N  .  .  .  .  .  .  = 
 =  .  =  =  =  =  =  =  .  = 
 =  .  =  .  .  .  .  C  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  C  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  N  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 
Solução Custo-uniforme (grafo) com custo 12:
['S', 'N', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'O']
Time:  0.2919459999999958
Podemos ver o rasto do plano
 =  =  =  =  =  =  =  =  =  = 
 =  @  +  +  +  +  +  +  +  = 
 =  +  =  =  =  =  =  =  +  = 
 =  .  =  .  .  .  .  *  +  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  =  .  C  .  .  .  .  = 
 =  .  =  .  .  .  .  .  .  = 
 =  .  .  N  .  .  .  .  .  = 
 =  .  .  .  .  .  .  .  D  = 
 =  =  =  =  =  =  =  =  =  = 


Indo uma casa para baixo para aumentar o valor da pastilha de crescimento e assim atingir o objetivo